# This is a sandbox for setting up a run of the simulation on `PRIVATE_AUTO` only. 

* It can be run on a small subsample pandas DataFrame
* The dask implementation might work with `map_partition()` but also might not. 


In [ ]:
import pandas as pd
import dask.dataframe as dd
import dask.array as da
import numpy as np
import replicaEVSE.load_curve as sim
import os
from dask.diagnostics import ProgressBar
import joblib

%reload_ext autoreload
%autoreload 2



pd.set_option('display.max_columns', None)

datadir = '../../data'



#Created in the EIA_data_download.ipynb notebook
existing_load=pd.read_csv('../../data/EIA_demand_summary.csv') 

# read in the joined trips and population data sets
merged_ddf = dd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips.parquet'))

# right now, only look at private auto trips
ddf = merged_ddf.loc[merged_ddf['mode'] == 'PRIVATE_AUTO']

# This is necessary for map partitions. Perhaps we save a new 
# version of the parquet files where this is true and 
# the data are presorted. The simulation does some sorting however.
# would this make it faster?
ddf = ddf.reset_index(drop=True)


# sort on person_id and start_time
#ddf = ddf.sort_values(by=['person_id', 'start_time', 'weekday']).reset_index(drop=True)

df = merged_ddf.head(10000)

small_ddf = dd.from_pandas(df, npartitions=10)

# to run the whole data set, uncomment the following line
# df = ddf.compute()


We can start a dask cluster to get diagnostic info. 

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=8)  # Launches a scheduler and workers locally
client = Client(cluster)  # Connect to distributed cluster and override default

pbar = ProgressBar()
pbar.register()

client

run the simulation on small dataset to test. 

In [ ]:
# this takes ~1min for 10,000 trips
out = sim.simulate_person_load(df, existing_load, 'base', managed=False)

There is a dask implementation that would be significantly faster or can do the embarrassingly parallel option below 

In [ ]:
# try to use dask
# not tested. 
# out = small_ddf.map_partitions(sim.simulate_person_load, existing_load, 'base', managed=False, align_dataframes=False)
# out.compute()

embarrassingly parallel

In [ ]:
number_of_chunks = 1000
df_list = np.array_split(df, number_of_chunks)

In [ ]:
# run the simulation in parallel
# df must be a pandas dataframe
charge_sims = joblib.Parallel(verbose=10, n_jobs=-1)(
    joblib.delayed(sim.simulate_person_load)(
    df=df,
    existing_load=existing_load,
    simulation_id='base',
    managed=False
) for df in df_list)

In [ ]:
# restack the dataframes
charges_list = [x['charges'] for x in charge_sims]
charges_df = dd.concat(charges_list)

loads_list = [x['loads'] for x in charge_sims]
loads_df = dd.concat(loads_list)

# below this is just exploring the data

In [ ]:
frac = 0.001 # ~2e4
thu = dd.read_parquet(os.path.join(datadir, 'northwest_2021_Q4_thursday_trip.parquet'))
sat = dd.read_parquet(os.path.join(datadir, 'northwest_2021_Q4_saturday_trip.parquet'))
pop = dd.read_parquet(os.path.join(datadir, 'northwest_2021_Q4_population.parquet'))
ddf = dd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips.parquet'))
# df = ddf.head(10000000)
# trips_ddf = dd.concat([thu, sat])
# ddf = dd.from_pandas(df, chunksize=1000)
# df['mode'].value_counts()

In [ ]:
len(trips_ddf.loc[trips_ddf['mode'] == 'PRIVATE_AUTO'])

In [ ]:
len(sat) + len(thu), len(ddf), len(pop)

In [ ]:
pop_len = 58.222322
trips_len = 159.624453 # million
join_len = 51.727268 
other_len = 49.674863
pop_len = 14.889896

In [ ]:
unique_people_in_pop_len = len(pop['person_id'].unique())


In [ ]:
trucks = trips_ddf.loc[trips_ddf['mode'] == 'COMMERCIAL']
other = trips_ddf.loc[trips_ddf['mode'] != 'PRIVATE_AUTO']

In [ ]:
len(other)

In [ ]:
trucks['person_id'].value_counts()

In [ ]:
trucks.loc[trucks['vehicle_type'] == 'MEDIUM_COMMERCIAL']['distance_miles'].hist(bins=100,  alpha=0.5)
trucks.loc[trucks['vehicle_type'] == 'HEAVY_COMMERCIAL']['distance_miles'].hist(bins=100, log=True, alpha=0.5)

In [ ]:
trips = dd.read_parquet(os.path.join(datadir, 'northwest_2021_Q4_thursday_trip.parquet'))